# Part One

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.pyplot import figure
import pyodbc 
import glob
import copy 

### Pulling Info From YML File

In [2]:
# Localise with . files 
# config_filename = 'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/ds41_42_config.yml'
config_filename = './ds41_42_config.yml'

In [3]:
config_filename

'./ds41_42_config.yml'

In [4]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [5]:
first_ID = config['inputs']['first_ID']

In [6]:
second_ID = config['inputs']['second_ID']

In [7]:
output_path = config['inputs']['output_path']

In [8]:
first_ID_datafiles = config[first_ID].values()

In [9]:
second_ID_datafiles = config[second_ID].values()

### Downloading SQL Data

In [10]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

In [11]:
query_all = "SELECT mgra, geotype, geozone FROM demographic_warehouse.dim.mgra WHERE series = 14 AND (geotype='cpa' OR geotype='jurisdiction')" #Remove the last and part when I do this for real 

In [12]:
sql_query = pd.read_sql_query(query_all,conn)

sql_df_all = pd.DataFrame(sql_query)

In [13]:
# SQl Data at different levels
jur_level = sql_df_all[sql_df_all['geotype']=='jurisdiction'].drop('geotype', axis=1)
cpa_level = sql_df_all[sql_df_all['geotype']=='cpa'].drop('geotype', axis=1)

### Downloading the first_ID Data

In [14]:
# first_ID Data Frame
first_ID_df = pd.DataFrame()
for file_name in first_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    first_ID_df = first_ID_df.append(working_df)

In [15]:
# Save the features_first_ID for future use (Used when creating the diff file)
features_first_ID = first_ID_df.drop(['mgra', 'year'], axis=1).columns

In [16]:
comparison_first_ID_no_geozone = copy.deepcopy(first_ID_df)

In [17]:
# Adding SQl Data to first_id_df
comparison_first_ID_processed_data = comparison_first_ID_no_geozone.merge(jur_level.drop_duplicates(), how='left', on='mgra')

In [18]:
# making it original
comparison_first_ID_processed_data.columns = [x + f'_{first_ID}' for x in comparison_first_ID_processed_data.columns]

In [19]:
comparison_first_ID_processed_data.head()

,mgra_DS35,taz_DS35,hs_DS35,hs_sf_DS35,hs_mf_DS35,hs_mh_DS35,hh_DS35,hh_sf_DS35,hh_mf_DS35,hh_mh_DS35,...,hotelroomtotal_DS35,luz_id_DS35,truckregiontype_DS35,district27_DS35,milestocoast_DS35,acres_DS35,effective_acres_DS35,land_acres_DS35,year_DS35,geozone_DS35
0,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
1,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
2,3,3358,52,52,0,0,52,52,0,0,...,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016,San Diego
3,4,3358,30,30,0,0,30,30,0,0,...,0,95,1,27,4.2782,7.976178,7.976178,7.976178,2016,San Diego
4,5,3358,28,28,0,0,28,28,0,0,...,0,95,1,27,4.0062,7.072502,7.063693,7.072502,2016,San Diego


In [20]:
#comparison_first_ID_processed_data.to_csv(f"{output_path}/comparison_{first_ID}_processed_data.csv")

### Downloading  the second_ID Data

In [21]:
# second_id Data Frame
second_ID_df = pd.DataFrame()
for file_name in second_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    second_ID_df = second_ID_df.append(working_df)

In [22]:
# Save the features of the second ID for future use (Used when creating the diff file)
features_second_ID = second_ID_df.drop(['mgra', 'year'], axis=1).columns

In [23]:
comparison_second_ID_no_geozone = copy.deepcopy(second_ID_df)

In [24]:
# Adding SQl Data to second_id_df
comparison_second_ID_processed_data = comparison_second_ID_no_geozone.merge(jur_level.drop_duplicates(), how='left', on='mgra')

In [25]:
# making it original
comparison_second_ID_processed_data.columns = [x + f'_{second_ID}' for x in comparison_second_ID_processed_data.columns]

In [26]:
comparison_second_ID_processed_data.head()

,mgra_DS41,taz_DS41,hs_DS41,hs_sf_DS41,hs_mf_DS41,hs_mh_DS41,hh_DS41,hh_sf_DS41,hh_mf_DS41,hh_mh_DS41,...,hotelroomtotal_DS41,luz_id_DS41,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41,year_DS41,geozone_DS41
0,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
1,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
2,3,3358,52,52,0,0,52,52,0,0,...,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016,San Diego
3,4,3358,30,30,0,0,30,30,0,0,...,0,95,1,27,4.2782,7.976178,7.976178,7.976178,2016,San Diego
4,5,3358,28,28,0,0,28,28,0,0,...,0,95,1,27,4.0062,7.072502,7.063693,7.072502,2016,San Diego


In [27]:
# comparison_second_ID_processed_data.to_csv(f"{output_path}/comparison_{second_ID}_processed_data.csv")

### Concatenate both DS dataframes

In [28]:
# Added geozone to merge keys to account for mgra's in multiple jurisdictions (or other geographical levels)
first_second_ID_comparison = comparison_first_ID_processed_data.merge(
    comparison_second_ID_processed_data,
    how='left',
    left_on=[f'mgra_{first_ID}',
             f'year_{first_ID}',
             f'geozone_{first_ID}'],
    right_on=[f'mgra_{second_ID}',
             f'year_{second_ID}',
             f'geozone_{second_ID}'])

In [29]:
# Clean green combined
first_second_ID_comparison = first_second_ID_comparison.drop([f'mgra_{second_ID}', f'year_{second_ID}', f'geozone_{second_ID}'], axis=1)
first_second_ID_comparison = first_second_ID_comparison.rename(columns={f'mgra_{first_ID}': 'mgra', f'year_{first_ID}': 'year', f'geozone_{first_ID}': 'geozone'})

In [30]:
# Because we're summing, if using series 14 data, mgra's in multiple jurisdictions will be counted multiple times
first_second_ID_comparison = first_second_ID_comparison.groupby(['mgra', 'year']).sum()

In [31]:
first_second_ID_comparison.head()

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
mgra year                                                                   
1    2016      3331       19          19           0           0       18   
     2018      3331       19          19           0           0       18   
     2020      3331       19          19           0           0       18   
     2023      3331       20          20           0           0       18   
     2025      3331       20          20           0           0       18   

           hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
mgra year                                                   ...   
1    2016          18           0           0            0  ...   
     2018          18           0           0            0  ...   
     2020          18           0           0            0  ...   
     2023          18           0           0            0  ...   
     2025          18           0           0            0  ...   

           midpriceroom_DS41  upscaleroom_DS41  hotelroomtotal_DS41  \
mgra year                                                             
1    2016                  0                 0                    0   
     2018                  0                 0                    0   
     2020                  0                 0                    0   
     2023                  0                 0                    0   
     2025                  0                 0                    0   

           luz_id_DS41  truckregiontype_DS41  district27_DS41  \
mgra year                                                       
1    2016           95                     1               27   
     2018           95                     1               27   
     2020           95                     1               27   
     2023           95                     1               27   
     2025           95                     1               27   

           milestocoast_DS41  acres_DS41  effective_acres_DS41  \
mgra year                                                        
1    2016             3.7997   16.615444             12.961482   
     2018             3.7997   16.615444             12.961482   
     2020             3.7997   16.615444             12.961482   
     2023             3.7997   16.615444             12.961482   
     2025             3.7997   16.615444             12.961482   

           land_acres_DS41  
mgra year                   
1    2016        16.615444  
     2018        16.615444  
     2020        16.615444  
     2023        16.615444  
     2025        16.615444  

[5 rows x 206 columns]

In [32]:
#first_second_ID_comparison.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison")

### Creating the complete Diff File

In [33]:
# Finding features common to both DSID data frames
list1_as_set = set(features_first_ID)
intersection = list1_as_set.intersection(features_second_ID)

shared_features = list(intersection)

In [48]:
shared_features[0:5]

['i7', 'hs_sf', 'mstallssam', 'emp_pvt_ed_post_k12_oth', 'i1']

In [49]:
# Calculate diff values between the two DS_ID's
diff_df = pd.DataFrame()

# NOTE: Subtracts second DS ID from first DS ID. If negative, then second DS ID was greater than first DS ID.
for column in shared_features:
    diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']

In [50]:
diff_df.head()

i7_diff  hs_sf_diff  mstallssam_diff  emp_pvt_ed_post_k12_oth_diff  \
mgra year                                                                       
1    2016        0           0                0                             0   
     2018        3           0                0                             0   
     2020        1           0                0                             0   
     2023        1           0                0                             0   
     2025       -2           0                0                             0   

           i1_diff  luxuryroom_diff  hh_mf_diff  enrollgrade9to12_diff  \
mgra year                                                                
1    2016        0                0           0                      0   
     2018        0                0           0                      0   
     2020       -1                0           0                      0   
     2023       -1                0           0                      0   
     2025       -1                0           0                      0   

           economyroom_diff  dstallssam_diff  ...  taz_diff  \
mgra year                                     ...             
1    2016                 0                0  ...         0   
     2018                 0                0  ...         0   
     2020                 0                0  ...         0   
     2023                 0                0  ...         0   
     2025                 0                0  ...         0   

           emp_prof_bus_svcs_diff  hhp_diff  emp_hotel_diff  \
mgra year                                                     
1    2016                       0         0               0   
     2018                       0         3               0   
     2020                       0         6               0   
     2023                       0         6               0   
     2025                       0        -2               0   

           enrollgradekto8_diff  budgetroom_diff  pop_diff  i5_diff  \
mgra year                                                             
1    2016                     0                0         0        0   
     2018                     0                0         3        0   
     2020                     0                0         6        0   
     2023                     0                0         6        0   
     2025                     0                0        -2        2   

           emp_pvt_ed_k12_diff  hs_mh_diff  
mgra year                                   
1    2016                    0           0  
     2018                    0           0  
     2020                    0           0  
     2023                    0           0  
     2025                    0           0  

[5 rows x 103 columns]

In [36]:
#diff_df.to_csv(f"{output_path}/{first_ID}_{second_ID}_complete_diff")

## CPA level Data

In [39]:
# Adding SQl Data to first_second_ID_combined dataframe
comparison_processed_data_cpa = first_second_ID_comparison.merge(cpa_level, how='left', on='mgra')

In [40]:
comparison_first_ID_no_geozone

,mgra,taz,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,...,upscaleroom,hotelroomtotal,luz_id,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,year
0,1,3331,19,19,0,0,18,18,0,0,...,0,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016
1,2,3331,35,35,0,0,34,34,0,0,...,0,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016
2,3,3358,52,52,0,0,52,52,0,0,...,0,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016
3,4,3358,30,30,0,0,30,30,0,0,...,0,0,95,1,27,4.2782,7.976178,7.976178,7.976178,2016
4,5,3358,28,28,0,0,28,28,0,0,...,0,0,95,1,27,4.0062,7.072502,7.063693,7.072502,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22997,22998,1290,92,92,0,0,87,87,0,0,...,0,0,14,1,1,2.6193,41.241522,26.863578,41.241522,2050
22998,22999,1290,0,0,0,0,0,0,0,0,...,0,0,14,1,1,2.3703,35.842780,30.635095,35.842780,2050
22999,23000,1290,131,131,0,0,126,126,0,0,...,0,0,14,1,1,2.1721,28.735275,16.109816,28.735275,2050
23000,23001,1254,85,85,0,0,82,82,0,0,...,0,0,14,1,1,2.7063,41.006144,28.126056,41.006144,2050


### first_ID df

In [41]:
# Adding SQl Data to first_id_df
comparison_first_ID_processed_data_cpa = comparison_first_ID_no_geozone.merge(cpa_level, how='left', on='mgra')

In [42]:
# Remove mgra column
comparison_first_ID_processed_data_cpa = comparison_first_ID_processed_data_cpa.drop('mgra', axis=1)

In [43]:
# Aggregate values by CPA and year (Included code to see "*Not in a CPA*" values)
comparison_first_ID_processed_data_cpa.groupby(['geozone', 'year']).sum()#.drop('*Not in a CPA*')

taz      hs   hs_sf   hs_mf  hs_mh      hh   hh_sf  \
geozone         year                                                            
*Not in a CPA*  2016  20407873  537680  335486  174259  27935  514815  321854   
                2018  20407873  543150  338232  176798  28120  519070  324456   
                2020  20407873  552179  341582  182477  28120  527086  327545   
                2023  20407873  566706  346152  192434  28120  539336  331057   
                2025  20407873  575094  349690  197284  28120  546883  334034   
...                        ...     ...     ...     ...    ...     ...     ...   
Via De La Valle 2032     57562     500     384     116      0     463     354   
                2035     57562     501     385     116      0     463     354   
                2040     57562     509     393     116      0     465     356   
                2045     57562     509     393     116      0     465     356   
                2050     57562     510     394     116      0     466     357   

                       hh_mf  hh_mh  gq_civ  ...  midpriceroom  upscaleroom  \
geozone         year                         ...                              
*Not in a CPA*  2016  166048  26913   17639  ...          4314         1485   
                2018  167621  26993   17858  ...          4712         1626   
                2020  172487  27054   17967  ...          4712         1626   
                2023  181180  27099   18437  ...          4844         1675   
                2025  185723  27126   18843  ...          4934         1706   
...                      ...    ...     ...  ...           ...          ...   
Via De La Valle 2032     109      0       0  ...             0            0   
                2035     109      0       0  ...             0            0   
                2040     109      0       0  ...             0            0   
                2045     109      0       0  ...             0            0   
                2050     109      0       0  ...             0            0   

                      hotelroomtotal  luz_id  truckregiontype  district27  \
geozone         year                                                        
*Not in a CPA*  2016           14300  771205             9833       52724   
                2018           15629  771205             9833       52724   
                2020           15629  771205             9833       52724   
                2023           16079  771205             9833       52724   
                2025           16378  771205             9833       52724   
...                              ...     ...              ...         ...   
Via De La Valle 2032               0    2471               32          64   
                2035               0    2471               32          64   
                2040               0    2471               32          64   
                2045               0    2471               32          64   
                2050               0    2471               32          64   

                      milestocoast          acres  effective_acres  \
geozone         year                                                 
*Not in a CPA*  2016    71746.9177  337262.563475    244582.594582   
                2018    71746.9177  337262.563475    244582.594582   
                2020    71746.9177  337262.563475    244582.594582   
                2023    71746.9177  337262.563475    244582.594582   
                2025    71746.9177  337262.563475    244582.594582   
...                            ...            ...              ...   
Via De La Valle 2032       51.7274     539.167396       423.495878   
                2035       51.7274     539.167396       423.495878   
                2040       51.7274     539.167396       423.495878   
                2045       51.7274     539.167396       423.495878   
                2050       51.7274     539.167396       423.495878   

                         land_ac

In [44]:
# Remove mgra column
comparison_processed_data_cpa = comparison_processed_data_cpa.drop('mgra', axis=1)

In [45]:
# Aggregate values by CPA and year (Included code to see "*Not in a CPA*" values)
comparison_processed_data_cpa = comparison_processed_data_cpa.groupby(['geozone', 'year']).sum().drop('*Not in a CPA*')

KeyError: 'year'

In [ ]:
comparison_processed_data_cpa.head()

In [ ]:
#comparison_processed_data_cpa.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_cpa")

## Jurisdiction level Data

In [ ]:
# Adding SQl Data to first_second_ID_combined dataframe
comparison_processed_data_jur = first_second_ID_combined.merge(jur_level, how='left', on='mgra')

In [ ]:
# Remove mgra column
comparison_processed_data_jur = comparison_processed_data_jur.drop('mgra', axis=1)

In [ ]:
# Aggregate values by Jurisdiction and year (Included code to see "*Not in a CPA*" values)
comparison_processed_data_jur = comparison_processed_data_jur.groupby(['geozone', 'year']).sum()

In [ ]:
comparison_processed_data_jur.head()

In [ ]:
#comparison_processed_data_jur.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_jur")

In [ ]:
# Unsure about file name, maybe spell out jurisdiction instead of jur?

In [ ]:
# Should we rename geozone column to CPA or jurisdiction?

## Region level Data

## Individual Level?

# Brainstorming for Part 2

In [ ]:
config['dof']['dof_data']

In [ ]:
first_second_ID_comparison.head()

In [ ]:
# MGRA level stats
first_second_ID_comparison.describe(percentiles=[0.5])